## Use Transformer-based Object Detection Model locally

In [ ]:
from transformers import pipeline
from PIL import Image, ImageDraw
import requests

obj_detector = pipeline("object-detection", model="facebook/detr-resnet-50")


In [ ]:
img_url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(img_url, stream=True).raw)
image

In [ ]:
results = obj_detector(img_url)

In [ ]:
results

In [ ]:
from PIL import Image, ImageDraw 

draw = ImageDraw.Draw(image)

for result in results:
    print(result['box'])
    draw.rectangle(list(result['box'].values()), outline ="red")
    draw.text(text=result['label'], xy=list(result['box'].values()))

In [ ]:
image

## Deploy to SageMaker Endpoint

Let's deploy a similar, but smaller model directly to a SageMaker endpoint - https://huggingface.co/hustvl/yolos-tiny

In [ ]:
import sagemaker
from sagemaker.huggingface.model import HuggingFaceModel

role = sagemaker.get_execution_role()

HF_MODEL_ID = "hustvl/yolos-tiny"

hub = {
    "HF_MODEL_ID": HF_MODEL_ID,
    "HF_TASK": "object-detection",
}

# creating SageMaker Model
huggingface_model = HuggingFaceModel(
    name='yolos-tiny',
    transformers_version="4.37",
    pytorch_version="2.1",
    py_version="py310",
    env=hub,
    role=role,
)

predictor = huggingface_model.deploy(
    instance_type='ml.g5.xlarge',
    initial_instance_count=1,
    endpoint_name='yolos-tiny',
    role=role,
    tags={},
    model_data_download_timeout=3600,
    container_startup_health_check_timeout=1200,
)

### Run Predictions

In [ ]:
predictor.predict({"inputs": "http://images.cocodataset.org/val2017/000000039769.jpg"})